In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("trabajo_final")
    .config("spark.sql.warehouse.dir", "hdfs:///user/warehouse")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!sh directorio.sh

Se ha creado el directorio warehouse/raw


In [4]:
!sh dir_hdfs.sh

El directorio /user/warehouse ya existe.
El directorio /user/warehouse/bronze ya existe.
El directorio /user/warehouse/silver ya existe.
El directorio /user/warehouse/gold ya existe.
Se crearon los directorios correctamente


In [6]:
!pip3 install shapely
!pip3 install geopandas

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [9]:
from pyspark import SparkFiles

# Crear el dataset crudo
!python3 01_gen_bd/create_data.py

file: str = "file:///workspace/warehouse/raw/water.parquet"
df = spark.read.parquet(file)
df.describe()

hdfs_path = "/user/warehouse/bronze/water_base"
df.write.mode("append").parquet(hdfs_path)

In [6]:
# !find / -name "water.parquet"

In [7]:
# !hdfs dfs -copyFromLocal warehouse/raw/water.parquet /user/warehouse/bronze/data_prueba_3.parquet

In [5]:
!hdfs dfs -ls /user/warehouse/

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-21 05:59 /user/warehouse/bronze
drwxr-xr-x   - root supergroup          0 2024-06-21 05:59 /user/warehouse/gold
drwxr-xr-x   - root supergroup          0 2024-06-21 05:59 /user/warehouse/silver


In [27]:
# hdfs_path = "hdfs:///user/warehouse/bronze/prueba2"

# spark.sql(f"CREATE DATABASE IF NOT EXISTS prueba2 LOCATION '{hdfs_path}';")

DataFrame[]

In [10]:
!hdfs dfs -ls /user/warehouse/bronze/

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-21 06:00 /user/warehouse/bronze/water_base


In [11]:
hdfs_path = "/user/warehouse/bronze/water_base"
df_bronce = spark.read.parquet(hdfs_path)
df_bronce.count()

9592

In [21]:
df_bronce.select("event_day").show()

+---------+
|event_day|
+---------+
|       05|
|       05|
|       05|
|       05|
|       05|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
|       04|
+---------+
only showing top 20 rows



In [36]:
from pyspark.sql.functions import avg, col, count, sum, min, max
from pyspark.sql.types import IntegerType

In [13]:
# Para Silver
# Cruce de información con la población
df_bronce = (
    df_bronce.withColumn("order_id", col("order_id").cast(IntegerType()))
    .withColumn("event_year", col("event_year").cast(IntegerType()))
    .withColumn("event_month", col("event_month").cast(IntegerType()))
    .withColumn("event_day", col("event_day").cast(IntegerType()))
    .withColumn("event_hour", col("event_hour").cast(IntegerType()))
    .withColumn("event_minute", col("event_minute").cast(IntegerType()))
    .withColumn("event_second", col("event_second").cast(IntegerType()))
    .withColumn("quantity_products", col("quantity_products").cast("int"))
    .withColumn("customer_id", col("customer_id").cast(IntegerType()))
)

In [6]:
df_bronce.printSchema()

root
 |-- partition_date: string (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- commune: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- employee_id: long (nullable = true)
 |-- event_date: string (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_month: integer (nullable = true)
 |-- event_second: integer (nullable = true)
 |-- event_year: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- quantity_products: integer (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [14]:
hdfs_path = "hdfs:///user/warehouse/silver/water"
bd_name = "water"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {bd_name} LOCATION '{hdfs_path}';")

DataFrame[]

In [15]:
!hdfs dfs -ls /user/warehouse/silver/

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-21 06:01 /user/warehouse/silver/water


In [17]:
bd_name = "water"
nombre_tabla = f"{bd_name}.data"
modo: str = "append"
df_bronce.write.mode(modo).saveAsTable(nombre_tabla)

In [18]:
!hdfs dfs -ls /user/warehouse/silver/water/data/

Found 4 items
-rw-r--r--   1 root supergroup          0 2024-06-21 06:02 /user/warehouse/silver/water/data/_SUCCESS
-rw-r--r--   1 root supergroup      90257 2024-06-21 06:02 /user/warehouse/silver/water/data/part-00000-fbfc6eef-0c28-449f-b209-88f936e06ae8-c000.snappy.parquet
-rw-r--r--   1 root supergroup      86587 2024-06-21 06:02 /user/warehouse/silver/water/data/part-00001-fbfc6eef-0c28-449f-b209-88f936e06ae8-c000.snappy.parquet
-rw-r--r--   1 root supergroup      83949 2024-06-21 06:02 /user/warehouse/silver/water/data/part-00002-fbfc6eef-0c28-449f-b209-88f936e06ae8-c000.snappy.parquet


In [19]:
spark.sql("SHOW TABLES IN water").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|   water|     data|      false|
+--------+---------+-----------+



In [20]:
spark.sql("SELECT * FROM water.data").show()

+--------------+--------+----------------+-----------+-----------+----------+---------+----------+------------+-----------+------------+----------+------------------+-----------------+--------------------+-----------------+-----------------+
|partition_date|order_id|         commune|customer_id|employee_id|event_date|event_day|event_hour|event_minute|event_month|event_second|event_year|          latitude|        longitude|        neighborhood|quantity_products|__index_level_0__|
+--------------+--------+----------------+-----------+-----------+----------+---------+----------+------------+-----------+------------+----------+------------------+-----------------+--------------------+-----------------+-----------------+
|      04042024|    null|        COMUNA 3|       6728|       9726|2024-04-04|        4|         0|           0|          4|           0|      2024|-75.54021272534422|6.264628115375797|            MANRIQUE|                3|             1983|
|      04042024|    null|       

In [21]:
# Para gold
df_gold = spark.read.table("water.data")

In [22]:
# tabla_totales = f"{bd_name}.pruebas"
df_gold.groupBy("commune", "neighborhood")\
    .agg(sum("quantity_products").alias("total_quantity"), count("*").alias("transactions"))\
    .orderBy("total_quantity", ascending=False).show()

+----------------+--------------------+--------------+------------+
|         commune|        neighborhood|total_quantity|transactions|
+----------------+--------------------+--------------+------------+
|       COMUNA 16|               BELÉN|          2095|         827|
|        COMUNA 7|             ROBLEDO|          2015|         789|
|        COMUNA 9|        BUENOS AIRES|          1851|         736|
|        COMUNA 3|            MANRIQUE|          1771|         698|
|       COMUNA 13|          SAN JAVIER|          1538|         606|
|        COMUNA 6|     DOCE DE OCTUBRE|          1520|         615|
|        COMUNA 4|            ARANJUEZ|          1394|         571|
|        COMUNA 8|       VILLA HERMOSA|          1353|         548|
|        COMUNA 1|             POPULAR|          1338|         530|
|CORREGIMIENTO 60|CORREGIMIENTO DE ...|          1199|         492|
|       COMUNA 14|          EL POBLADO|          1112|         456|
|        COMUNA 5|            CASTILLA|         

In [23]:
!hdfs dfs -ls /user/warehouse/gold/

In [26]:
hdfs_path = "hdfs:///user/warehouse/gold/water_db"
bd_name = "water_db"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {bd_name} LOCATION '{hdfs_path}';")

DataFrame[]

In [27]:
!hdfs dfs -ls /user/warehouse/gold/

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-21 06:03 /user/warehouse/gold/water_db


In [29]:
!hdfs dfs -ls /user/warehouse/gold/water_db/

In [37]:
bd_name = "water_db"
tabla_totales = f"{bd_name}.data"

mode="append"
df_gold.groupBy("commune", "neighborhood", "event_date")\
    .agg(
        sum("quantity_products").alias("total_ventas"),
        count("*").alias("numero_transacciones"),
        avg("quantity_products").alias("promedio_ventas"),
        max("quantity_products").alias("maximo_numero_ventas"),
        min("quantity_products").alias("minimo_numero_ventas")
    )\
    .write\
    .bucketBy(10, "event_date", "commune") \
    .mode(mode).saveAsTable(tabla_totales)
    # .orderBy("total_ventas", ascending=False).show(1)

In [40]:
!hdfs dfs -ls /user/warehouse/gold/water_db/data

Found 66 items
-rw-r--r--   1 root supergroup          0 2024-06-21 06:10 /user/warehouse/gold/water_db/data/_SUCCESS
-rw-r--r--   1 root supergroup       2349 2024-06-21 06:10 /user/warehouse/gold/water_db/data/part-00002-5bcbefd3-f50c-4df5-87af-8b6e7dcba050_00000.c000.snappy.parquet
-rw-r--r--   1 root supergroup       2736 2024-06-21 06:10 /user/warehouse/gold/water_db/data/part-00004-5bcbefd3-f50c-4df5-87af-8b6e7dcba050_00009.c000.snappy.parquet
-rw-r--r--   1 root supergroup       2340 2024-06-21 06:10 /user/warehouse/gold/water_db/data/part-00005-5bcbefd3-f50c-4df5-87af-8b6e7dcba050_00007.c000.snappy.parquet
-rw-r--r--   1 root supergroup       2394 2024-06-21 06:10 /user/warehouse/gold/water_db/data/part-00005-5bcbefd3-f50c-4df5-87af-8b6e7dcba050_00008.c000.snappy.parquet
-rw-r--r--   1 root supergroup       2376 2024-06-21 06:10 /user/warehouse/gold/water_db/data/part-00011-5bcbefd3-f50c-4df5-87af-8b6e7dcba050_00004.c000.snappy.parquet
-rw-r--r--   1 root supergroup       2583 

In [ ]:
# http://localhost:50070/webhdfs/v1/user/warehouse/gold/water_db/data/data?op=OPEN

In [ ]:
http://<localhost>:<50070>/webhdfs/v1/<user/warehouse/gold>?op=OPEN

In [42]:
!hdfs dfs -ls /user/warehouse/gold/water_db

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-21 06:10 /user/warehouse/gold/water_db/data


In [ ]:
http://172.17.0.2:50070/webhdfs/v1/user/warehouse/gold/water_db?op=OPEN